In [1]:
import pandas as pd
import pickle
import joblib
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================
# 1. 경로 설정 (파일이 있는 폴더 경로로 수정하세요)
# ==========================================
base_path = "./"  # 파일이 현재 같은 폴더에 있다면 "./"
# 만약 다른 폴더면: base_path = "ethereum_analysis/model_results/..."

print("🔍 [1단계] 사용된 Feature 이름 전수 조사 (selected_features.pkl)")
try:
    with open(os.path.join(base_path, 'selected_features.pkl'), 'rb') as f:
        features = pickle.load(f)
    
    print(f"총 Feature 개수: {len(features)}")
    
    # 🚨 금지어 리스트: 미래를 암시하는 단어들
    banned_words = ['next', 'future', 'target', 'label', 'close_-1', 'direction']
    suspicious_features = [col for col in features if any(bad in col.lower() for bad in banned_words)]
    
    if suspicious_features:
        print("\n🔥🔥 [치명적 경고] 누수 의심 변수가 발견되었습니다! 🔥🔥")
        print("이 변수들이 X(입력)에 포함되어 있습니다. 무조건 제거해야 합니다:")
        for feat in suspicious_features:
            print(f" - {feat}")
    else:
        print("✅ 변수 이름상으로는 명백한 미래 참조(next_*) 변수가 보이지 않습니다.")
        
except Exception as e:
    print(f"파일 열기 실패: {e}")

print("\n" + "="*50 + "\n")

print("🔍 [2단계] 모델이 중요하게 생각한 변수 확인 (XGBoost.pkl)")
try:
    # XGBoost나 LightGBM 모델 파일을 엽니다
    model_path = os.path.join(base_path, 'XGBoost.pkl') 
    if not os.path.exists(model_path):
        model_path = os.path.join(base_path, 'LightGBM.pkl') # 없으면 LightGBM 시도
        
    with open(model_path, 'rb') as f:
        model = joblib.load(f) # 또는 pickle.load(f)
    
    # Feature Importance 추출
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        
        # DataFrame으로 변환
        fi_df = pd.DataFrame({'feature': features, 'importance': importances})
        fi_df = fi_df.sort_values(by='importance', ascending=False).head(10)
        
        print("\n📊 Top 10 중요 변수 (이 중 하나가 압도적으로 높으면 누수입니다):")
        print(fi_df)
        
        # 가장 중요한 변수의 비중이 50%를 넘는지 체크
        top_score = fi_df.iloc[0]['importance']
        if top_score > 0.5: # 전체 중요도의 50% 이상을 혼자 차지
            print(f"\n🔥🔥 [경고] '{fi_df.iloc[0]['feature']}' 변수 혼자서 중요도 {top_score:.2f}를 차지합니다.")
            print("단일 변수가 예측을 다 해먹고 있다는 뜻으로, 강력한 누수 신호입니다.")
            
    else:
        print("모델에서 feature_importances_ 속성을 찾을 수 없습니다.")

except Exception as e:
    print(f"모델 파일 분석 실패: {e}")

print("\n" + "="*50 + "\n")

print("🔍 [3단계] 백테스트 성과 확인 (fold_summary.csv)")
try:
    summary_df = pd.read_csv(os.path.join(base_path, 'fold_summary.csv'))
    
    # Accuracy나 Precision이 비정상적으로 높은지 확인
    print("\n📊 Fold 별 성능 요약:")
    if 'accuracy' in summary_df.columns:
        print(summary_df[['fold', 'accuracy', 'precision', 'recall']])
        
        avg_acc = summary_df['accuracy'].mean()
        if avg_acc > 0.8:
            print(f"\n🔥🔥 [의심] 평균 정확도가 {avg_acc:.2f}입니다.")
            print("금융 시장에서 80% 이상의 정확도는 사실상 불가능합니다. 미래 데이터 참조가 확실시됩니다.")
    else:
        print(summary_df.head())

except Exception as e:
    print(f"CSV 열기 실패: {e}")


🔍 [1단계] 사용된 Feature 이름 전수 조사 (selected_features.pkl)
총 Feature 개수: 30
✅ 변수 이름상으로는 명백한 미래 참조(next_*) 변수가 보이지 않습니다.


🔍 [2단계] 모델이 중요하게 생각한 변수 확인 (XGBoost.pkl)

📊 Top 10 중요 변수 (이 중 하나가 압도적으로 높으면 누수입니다):
                    feature  importance
1                    DPO_20    0.318962
22                    ITS_9    0.070402
11                 RV_RATIO    0.069054
9                 gold_GOLD    0.065628
16           ROLLING_MIN_20    0.049724
3                     ISA_9    0.049116
5                    IKS_26    0.048734
10  curve_curve-dex_eth_tvl    0.047814
2      eth_btc_corr_highvol    0.041888
25                  dxy_DXY    0.039726


🔍 [3단계] 백테스트 성과 확인 (fold_summary.csv)

📊 Fold 별 성능 요약:
                Model  Train_Accuracy  Train_Precision  Train_Recall  \
0        RandomForest         0.83250         0.744186      0.847682   
1            LightGBM         0.89875         0.835866      0.910596   
2             XGBoost         0.84250         0.755814      0.860927   
3  LogisticRegr